In [ ]:
import requests
from bs4 import BeautifulSoup
import random
import time
import pandas as pd

url = "https://www.cricbuzz.com/cricket-series/7476/icc-mens-t20-world-cup-2024"

user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:117.0) Gecko/20100101 Firefox/117.0",
    "Mozilla/5.0 (Linux; Android 12; Pixel 6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Mobile Safari/537.36"
]

headers = {
    "User-Agent": random.choice(user_agents),
    "Accept-Language": "en-US,en;q=0.9",
    "Referer": "https://www.google.com/"
}

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, "html.parser")

# Get all player links
players = soup.find_all("a", {"class": "cb-col pad5 cb-col-100"})
base_url = "https://www.cricbuzz.com"

player_data = []

for player in players:
    player_name = player.text.strip()
    player_link = base_url + player['href']

    res = requests.get(player_link, headers=headers)
    psoup = BeautifulSoup(res.text, "html.parser")

    # Extract team name
    team_tag = psoup.find("h3", {"class": "cb-font-18 text-gray"})
    team_name = team_tag.text.strip() if team_tag else "Unknown"

    # Extract player image
    img_tag = psoup.find("img", {"title": "profile image"})
    img_url = img_tag['src'] if img_tag else "No image"

    # Extract DOB, Birthplace, Batting & Bowling styles
    info_tags = psoup.find_all("div", {"class": "cb-col cb-col-60 cb-lst-itm-sm"})
    
    dob = info_tags[0].text.strip() if len(info_tags) > 0 else "Unknown"
    birth_place = info_tags[1].text.strip() if len(info_tags) > 1 else "Unknown"
    batting_style = info_tags[2].text.strip() if len(info_tags) > 2 else "Unknown"
    role = info_tags[3].text.strip() if len(info_tags) > 3 else "Unknown"
    bowling_style = info_tags[4].text.strip() if len(info_tags) > 4 else "Unknown"

    player_data.append({
        "player": player_name,
        "team": team_name,
        "dob": dob,
        "birth_place": birth_place,
        "batting": batting_style,
        "role":role,
        "bowling": bowling_style,
        "image": img_url,
        "profile_link": player_link
    })

    print(f"{player_name} | {team_name} | {dob} | {birth_place} | {batting_style} |{role}| {bowling_style} | {img_url}")

    # sleep to avoid being blocked
    time.sleep(1)

# Optional: save to JSON
# import json
# with open("players.json", "w") as f:
#     json.dump(player_data, f, indent=2)
df = pd.DataFrame(player_data)
df.to_excel("t20_worldcup_players.xlsx", index=False)
print("Data saved to t20_worldcup_players.xlsx")